# Linked stack of any element type

<div class="alert alert-block alert-info">
    You can find all of the C programs in this notebook in the subdirectory containing this notebook:
    <code>./src/qlgstack</code>
</div>

A disadvantage of our stack implementation is that it is limited to holding elements that are compatible with
`int`. If we required a stack of some other type, then we would have to create a new stack type (perhaps by
copying and renaming our `qlstack` type) which leads to code duplication.

Many programming languages provide support for *generic* types where the user can specify runtime type information.
For example, in Java, the programmer can create stack objects capable of holding elements of specified types:

```java
// import java.util.Stack;

Stack<Integer> iStack = new Stack<>();    // stack of Integer
Stack<Double>  dStack = new Stack<>();    // stack of Double
Stack<MyType>  mStack = new Stack<>();    // stack of MyType
```

In languages that support generic types, the programmer only implements the type once thus avoiding code duplication.

Java's generic type mechanism is type safe. Creating a `Stack<Integer>` object creates a stack capable of holding
only `Integer`-type objects.

Dynamically typed languages such as Python typically provide collections that can hold instances of any type.
For example, the programmer can add a string, a number, and an instance of any other type to the same list.
It is the programmer's responsibility to remember or determine the actual type of each list element.

This notebook describes how to create a stack in C that can hold elements of any type (similar to a Python
collection). 

## A node that stores a pointer to an element

Consider the node struct from our stack-of-int implementation:

```c
struct node {
    int elem;
    struct node *next;
};
```

In the above implementation, a node stores a copy of an `int` element.

C has no type that can store the value of an arbitrary type. However, it does have the void pointer type:

```c
void *
```

which is a pointer to a value of *an unspecified type*. A void pointer variable can store the value of any
pointer (without a cast). Thus, to create a stack that can hold any element type, we can require that
users push pointers to elements onto the stack. The nodes of the stack store the pointers to elements instead
of the elements themselves:

```c
struct node {
    void *elem;
    struct node *next;
};
```

An important consequence of storing pointers to elements is that each element must have a lifetime that ends
at or after the stack's lifetime. Typically, this means that memory for each element will be dynamically
allocated.

## A linked stack that stores pointers to elements

To represent our generic stack, we forward declare an opaque type `struct qglstack_tag` and a `typedef` in the
header file:

```c
// in qlgstack.h

typedef struct qlgstack_tag qlgstack;
```

In the source code file we declare the node and stack structs:

```c
// in qlgstack.c

struct node {
    int elem;
    struct node *next;
};

struct qlstack_tag {
    struct node *top;
    size_t size;
};
```

The push operation requires that the caller pass a pointer to the pushed element:

```c
/*
 * Pushes a pointer to the specified element onto the top of the stack pointed
 * at by s.
 * Returns s, or a null pointer if s is a null pointer.
 */
qlgstack *qlgstack_push(qlgstack *s, void *elem);
```

The pop operation writes a pointer to the poppped element into a pointer object provided by the caller:

```c
/*
 * Pops the top element off the top of the stack pointed
 * at by s. If popped is not null then a pointer to the popped element is written
 * into the object pointed at by popped.
 * Returns s, or a null pointer if s is a null pointer.
 */
qlgstack *qlgstack_pop(qlgstack *s, void **popped);
```

The implementations of the push and pop operations are almost identical to the corresponding functions in `qlstack`:

```c
qlgstack *qlgstack_push(qlgstack *s, void *elem) {
    if (!s) {
        return NULL;
    }
    struct node *n = malloc(sizeof(struct node));
    if (!n) {
        return NULL;
    }
    n->elem = elem;
    n->next = s->top;
    s->top = n;
    s->size++;
    return s;
}

qlgstack *qlgstack_pop(qlgstack *s, void **popped) {
    if (!s) {
        return NULL;
    }
    struct node *oldtop = s->top;
    void *elem = oldtop->elem;
    s->top = oldtop->next;
    s->size--;
    free(oldtop);
    if (popped) {
        *popped = elem;
    }
    return s;
}
```

## Some problems

Examining the functions that print and free a stack reveals some problems that must be solved.

In `qlstack`, we print each element of the stack by using the `printf` function. If `n` is a pointer to a
`node` then printing the element is as simple as:

```c
printf("%d", n->elem);
```

In our generic stack, the stack does not know the actual element type; thus, there is no way for the function
to print the element.

When deallocation memory used by a stack, it may (or may not) be the case that the user wants the memory for
each element to also be deallocated. Note that in many use cases, the user *really* wants the stack to deallocate
memory used by the elements because the stack is the only object holding pointers to the elements.
Because our stack does not know the actual element type, there is no way
for the deallocation function to correctly deallocate memory used by the elements.

A solution to both problems is to realize that printing an element or deallocating memory for an element can
be encapsulated in a function *known to the user of the stack*. For example, to print an element of some
user-defined type, the user should be capable of providing a function resembling:

```c
void print_my_user_defined_type(const void *elem)
```

where `elem` is a pointer to an object of the user-defined type.

Similarly, to free an element of some user-defined type, the user should be capable of providing a function resembling:

```c
void free_my_user_defined_type(void *elem)
```

where `elem` is a pointer to an object of the user-defined type. Note that `elem` is not declared as being
`const` because in the most common case, the deallocation function will end up calling `free(elem)` at some
point, and the `free` function expects a non-`const` argument.

Therefore, we could implement the printing and deallocating functions for our stack if there were a way to
pass a function to another function. In C, a function can be passed to another function by passing a pointer
to a function.

## Pointer to function

Recall that a pointer to a data type *T* is simply an object that stores the memory address of an object of
type *T*. For example, a pointer to `int` is simply an object that stores the address of an `int` object.

```c
int x = 1;
int *p = &x;   // p stores the address of x
```

A pointer to a function is an object that stores the address of a function. To declare a pointer to a function,
you simply take the function declaration and replace the function name with:

```
(*your_ptr_variable_name)
```

For example, consider the following function declaration:

```c
void print_my_user_defined_type(const void *elem);
```

To create a pointer variable named `ptr2func` to `print_my_user_defined_type` simply repeat the function
declaration replacing the function name with `(*ptr2func)`:

```c
void (*ptr2func)(const void *elem);
```

The pointer variable declaration says that `ptr2func` is a pointer to a function that returns `void` and
has the parameter list `(const void *)`. To assign a value to `ptr2func`, simply take the address of
a suitable function:

```c
ptr2func = &print_my_user_defined_type;
```

The use of the address-of operation `&` is not strictly required; the following is also acceptable:

```c
ptr2func = &print_my_user_defined_type;
```

Our pointer can point at any function that returns `void` and has `const void *` as its only parameter. For example,
suppose that we had the function:

```c
void print_int(const void *elem) { /* not shown */ }
```

Then we could write:

```c
ptr2func = &print_int;
```

Given a pointer to a function, the pointed at function can be called by dereferencing the pointer. For example,
we can print an `int` value using `ptr2func` like so:

```c
ptr2func = &print_int;

int a = 5;
(*ptr2func)(&a);             // dereference ptr2func and pass address of a to the function
```

Explicitly dereferencing the pointer is not strictly required; simply treating the pointer name as the function
name is also acceptable:

```c
ptr2func = &print_int;

int a = 5;
ptr2func(&a);               // implicitly dereference ptr2func and pass address of a to the function
```

The following program uses a pointer to a function to print an `int`, `double`, and string:

In [ ]:
#include <stdio.h>

void print_int(const void *elem) {
    const int *p = elem;     // convert elem to pointer to int
    int val = *p;            // dereference p to get the int value
    printf("%d", val);       // print val
}

void print_double(const void *elem) {
    const double *p = elem;  // convert elem to pointer to double
    double val = *p;         // dereference p to get the double value
    printf("%f", val);       // print val
}

void print_string(const void *elem) {
    const char *p = elem;     // convert elem to pointer to char
    printf("%s", p);          // print object pointed at by p as a string
}

int main(void) {
    // a pointer to one of the print_* functions
    void (*ptr2func)(const void *elem) = NULL;
    
    int x = 1;
    ptr2func = &print_int;
    (*ptr2func)(&x);
    printf("\n");
    
    double y = -99.9;
    ptr2func = &print_double;
    (*ptr2func)(&y);
    printf("\n");
    
    char str[] = "CISC220";
    ptr2func = &print_string;
    (*ptr2func)(str);
    printf("\n");
    
    return 0;
}

## Pointer to function and `typedef`

A `typedef` can be used to simplify using pointers to functions. To create a `typedef` of a pointer-to-function
type, start by writing `typedef` followed by a function declaration that you want to create a pointer to.
For example, consider the following function declaration:

```c
void print_my_user_defined_type(const void *elem);
```

To create a simple name for a pointer-to-function type, start by writing:

```c
typedef void print_my_user_defined_type(const void *elem);
```

Next, replace the function name with `(*your_alias)` where `your_alias` is the alias for the pointer type. For
example:

```c
typedef void (*print_func)(const void *elem);
```

The above `typedef` says that `print_func` is an alternate name for a pointer type that points at a function
that returns `void` and has `const void *` as its only parameter.

Given the `typedef` we can create and assign a pointer like so:

```c
print_func p = &print_my_user_defined_type;
```

Notice that writing `print_func p` is much easier to write (and read) than `void (*p)(const void *elem)`. 

## Printing a stack

For our stack implementation, we require a pointer type that points at a function that can print an element.
In the header file, we provide a `typedef` for such a pointer type:

```c
// in qlgstack.h

/*
 * qlgstack_print_func is the alias for a pointer to a function that
 * prints an element pointed at by elem.
 */
typedef void (*qlgstack_print_func)(const void *elem);
```

In the header file, we also provide the following forward declaration of the function that prints a stack:

```c
/*
 * Prints a representation of the specified stack to standard out
 * with an optional preamble and postscript. Each element is printed
 * using the function pointed at by ptr.
 * If pre is not null, then the string pre is printed before the
 * stack representation.
 * If post is not null, then the string post is printed after the
 * stack representation.
 * If ptr is null then each element is printed as a question mark.
 * Undefined behavior if s is a null pointer.
 */
void qlgstack_print(const char *pre, const qlgstack *s, const char *post,
                    qlgstack_print_func ptr);
```

This function declaration is the same as the function in our stack-of-integer implementation except that
it also includes a pointer to a function that print the user's element type. The implementation of
`qlgstack_print` is shown below:

```c
/*
 * Prints an element as a question mark. The default element printing function.
 */
void print_default_elem(const void *elem) {
    printf("?");
}

void qlgstack_print(const char *pre, const qlgstack *s, const char *post,
                    qlgstack_print_func ptr) {
    if (!ptr) {
        ptr = print_default_elem;        // print element as ?
    }
    if (pre) {
        printf("%s", pre);
    }
    printf("top : ");
    if (!qlgstack_is_empty(s)) {
        struct node *n = s->top;
        ptr(n->elem);                    // printf("%d", n->elem); in qlstack
        n = n->next;
        while (n) {
            printf(", ");                //
            ptr(n->elem);                // printf("%d", n->elem); in qlstack
            n = n->next;
        }
    }
    printf(" : bottom");
    if (post) {
        printf("%s", post);
    }
}
```

Notice that the only changes to the function are those related to printing an individual element.

## Deallocating a stack

If the user wants the stack to deallocate memory used by the elements then a function that encapsulates the
deallocation details is required. In the header file, we provide a `typedef` for such a pointer type:

```c
// in qlgstack.h

/*
 * qlgstack_free_func is the alias for a pointer to a function that
 * deallocates memory used by an element pointed at by elem.
 */
typedef void (*qlgstack_free_func)(void *elem);
```

In the header file, we also provide the following forward declaration of the function that deallocates
memory used by a stack:

```c
/*
 * Deallocates memory allocated for the stack pointed at by s.
 * If ptr is not NULL, then memory for each element is deallocated by
 * calling the function pointed at by ptr.
 * Does nothing if s is NULL.
 */
void qlgstack_free(qlgstack *s,
                    qlgstack_free_func ptr);
```

This function declaration is the same as the function in our stack-of-integer implementation except that
it also includes a pointer to a function that deallocates memory used by the user's element type. 
The implementation of `qlgstack_free` is shown below:

```c
void qlgstack_free(qlgstack *s,
                    qlgstack_free_func ptr) {
    if (!s) {
        return;
    }
    // free all nodes
    struct node *n = s->top;
    while (n) {
        struct node *next = n->next;
        if (ptr) {
            ptr(n->elem);           // deallocate memory used by the element
        }
        free(n);
        n = next;
    }
    // free the stack
    free(s);
}
```

Notice that the only change to the function is the function call to deallocate memory used by the element.

## `demo.c`

A short program that demonstrates the functionality of our stack is shown below:

```c
// demo.c

#include <stdio.h>
#include <stdlib.h>
#include "qlgstack.h"

void print_int(const void *elem) {
    const int *val = elem;
    printf("%d", *val);
}

void free_int(void *elem) {
    free(elem);
}

int main(int argc, char *argv[]) {
    size_t n = argc - 1;
    qlgstack *q1 = qlgstack_new();
    qlgstack *q2 = qlgstack_new();

    for (size_t i = 0; i < n; i++) {
        int *val1 = malloc(sizeof(int));
        int *val2 = malloc(sizeof(int));
        *val1 = atoi(argv[i + 1]);
        *val2 = -*val1;
        if (!qlgstack_push(q1, val1)) {
            fprintf(stderr, "error pushing to q1\n");
            qlgstack_free(q1, free_int);
            exit(EXIT_FAILURE);
        }
        if (!qlgstack_push(q2, val2)) {
            fprintf(stderr, "error pushing to q2\n");
            qlgstack_free(q1, free_int);
            exit(EXIT_FAILURE);
        }
        qlgstack_print("q1 ", q1, "\n", print_int);
        qlgstack_print("q2 ", q2, "\n", print_int);
    }

    for (size_t i = 0; i < n; i++) {
        int *val1;
        int *val2;
        char pre[100];
        qlgstack_pop(q1, &val1);
        sprintf(pre, "popped %d, q1 ", *val1);
        qlgstack_print(pre, q1, "\n", print_int);
        
        qlgstack_pop(q2, &val2);
        sprintf(pre, "popped %d, q2 ", *val2);
        qlgstack_print(pre, q2, "\n", print_int);
    }

    qlgstack_free(q1, free_int);
    qlgstack_free(q2, free_int);

    return 0;
}
```

The program can be compiled using the following command:

```
gcc -o demo demo.c qlstack.c
```

The program pushes integer elements provided on the command line onto a pair of stacks, and then pops the
elements from the stacks until the stacks are empty. The program can be run from the command line like so:

```
./demo 1 2 3
```